In [4]:
import numpy as np

# 1. Definicja tekstu wejściowego
data = ("The various sub-fields of AI research are centered around particular goals and the use of particular tools. "
        "The traditional goals of AI research include reasoning, knowledge representation, planning, learning, "
        "natural language processing, perception, and the ability to move and manipulate objects")

# Przetwarzanie danych
chars = list(set(data))
data_size, X_size = len(data), len(chars)
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

# 2. Hiperparametry dla osiągnięcia wysokiej dokładności
H_size = 128         
T_steps = 25         
learning_rate = 1e-1
weight_sd = 0.1
z_size = H_size + X_size

# 3. Funkcje aktywacji
def sigmoid(x): return 1 / (1 + np.exp(-x))
def dsigmoid(y): return y * (1 - y)
def tanh(x): return np.tanh(x)
def dtanh(y): return 1 - y * y

# 4. Klasy parametrów
class Param:
    def __init__(self, name, value):
        self.name = name
        self.v = value
        self.d = np.zeros_like(value)
        self.m = np.zeros_like(value) # AdaGrad

class Parameters:
    def __init__(self):
        self.W_f = Param('W_f', np.random.randn(H_size, z_size) * weight_sd + 0.5)
        self.b_f = Param('b_f', np.zeros((H_size, 1)))
        self.W_i = Param('W_i', np.random.randn(H_size, z_size) * weight_sd + 0.5)
        self.b_i = Param('b_i', np.zeros((H_size, 1)))
        self.W_C = Param('W_C', np.random.randn(H_size, z_size) * weight_sd)
        self.b_C = Param('b_C', np.zeros((H_size, 1)))
        self.W_o = Param('W_o', np.random.randn(H_size, z_size) * weight_sd + 0.5)
        self.b_o = Param('b_o', np.zeros((H_size, 1)))
        self.W_v = Param('W_v', np.random.randn(X_size, H_size) * weight_sd)
        self.b_v = Param('b_v', np.zeros((X_size, 1)))
        
    def all(self):
        return [self.W_f, self.W_i, self.W_C, self.W_o, self.W_v,
               self.b_f, self.b_i, self.b_C, self.b_o, self.b_v]

# 5. Forward Pass, Backward Pass i Trening
def forward(x, h_prev, C_prev, p):
    z = np.row_stack((h_prev, x))
    f = sigmoid(np.dot(p.W_f.v, z) + p.b_f.v)
    i = sigmoid(np.dot(p.W_i.v, z) + p.b_i.v)
    C_bar = tanh(np.dot(p.W_C.v, z) + p.b_C.v)
    C = f * C_prev + i * C_bar
    o = sigmoid(np.dot(p.W_o.v, z) + p.b_o.v)
    h = o * tanh(C)
    v = np.dot(p.W_v.v, h) + p.b_v.v
    y = np.exp(v) / np.sum(np.exp(v))
    return z, f, i, C_bar, C, o, h, v, y

def backward(target, dh_next, dC_next, C_prev, z, f, i, C_bar, C, o, h, v, y, p):
    dv = np.copy(y)
    dv[target] -= 1
    p.W_v.d += np.dot(dv, h.T)
    p.b_v.d += dv
    dh = np.dot(p.W_v.v.T, dv) + dh_next
    do = dh * tanh(C)
    do = dsigmoid(o) * do
    p.W_o.d += np.dot(do, z.T)
    p.b_o.d += do
    dC = dh * o * dtanh(tanh(C)) + dC_next
    dC_bar = dC * i
    dC_bar = dtanh(C_bar) * dC_bar
    p.W_C.d += np.dot(dC_bar, z.T)
    p.b_C.d += dC_bar
    di = dC * C_bar
    di = dsigmoid(i) * di
    p.W_i.d += np.dot(di, z.T)
    p.b_i.d += di
    df = dC * C_prev
    df = dsigmoid(f) * df
    p.W_f.d += np.dot(df, z.T)
    p.b_f.d += df
    dz = (np.dot(p.W_f.v.T, df) + np.dot(p.W_i.v.T, di) + 
          np.dot(p.W_C.v.T, dC_bar) + np.dot(p.W_o.v.T, do))
    dh_prev = dz[:H_size, :]
    dC_prev = f * dC
    return dh_prev, dC_prev

# 6. Pętla treningowa
p = Parameters()
n, pointer = 0, 0
h_prev = np.zeros((H_size, 1))
C_prev = np.zeros((H_size, 1))

while True:
    if pointer + T_steps >= len(data) or n == 0:
        h_prev = np.zeros((H_size, 1))
        C_prev = np.zeros((H_size, 1))
        pointer = 0

    inputs = [char_to_idx[ch] for ch in data[pointer:pointer+T_steps]]
    targets = [char_to_idx[ch] for ch in data[pointer+1:pointer+T_steps+1]]

    # Forward
    z_s, f_s, i_s, C_bar_s, C_s, o_s, h_s, v_s, y_s = {}, {}, {}, {}, {}, {}, {}, {}, {}
    C_s[-1] = np.copy(C_prev)
    h_s[-1] = np.copy(h_prev)
    loss = 0
    for t in range(len(inputs)):
        x = np.zeros((X_size, 1))
        x[inputs[t]] = 1
        z_s[t], f_s[t], i_s[t], C_bar_s[t], C_s[t], o_s[t], h_s[t], v_s[t], y_s[t] = forward(x, h_s[t-1], C_s[t-1], p)
        loss += -np.log(y_s[t][targets[t], 0])

    # Backward
    for param in p.all(): param.d = np.zeros_like(param.v)
    dh_next = np.zeros_like(h_s[0])
    dC_next = np.zeros_like(C_s[0])
    for t in reversed(range(len(inputs))):
        dh_next, dC_next = backward(targets[t], dh_next, dC_next, C_s[t-1], z_s[t], f_s[t], i_s[t], C_bar_s[t], C_s[t], o_s[t], h_s[t], v_s[t], y_s[t], p)

    # Aktualizacja parametrów (AdaGrad)
    for param in p.all():
        param.m += param.d * param.d
        param.v += -learning_rate * param.d / np.sqrt(param.m + 1e-8)

    # Raportowanie postępu
    if n % 500 == 0:
        print(f'Iteracja {n}, Strata: {loss:.4f}')
    
    # Warunek stopu 
    if loss < 0.1:
        print(f"Osiągnięto wymaganą dokładność (strata: {loss:.4f}) w iteracji {n}")
        break

    pointer += T_steps
    n += 1

C:\Users\Dawid\AppData\Local\Temp\ipykernel_12752\679877971.py:54: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  z = np.row_stack((h_prev, x))


Iteracja 0, Strata: 84.9642
Iteracja 500, Strata: 30.1067
Iteracja 1000, Strata: 27.5497
Iteracja 1500, Strata: 14.5938
Iteracja 2000, Strata: 6.4958
Iteracja 2500, Strata: 3.7164
Iteracja 3000, Strata: 1.9775
Iteracja 3500, Strata: 2.8534
Iteracja 4000, Strata: 2.5176
Iteracja 4500, Strata: 3.6551
Iteracja 5000, Strata: 8.0607
Iteracja 5500, Strata: 9.5764
Iteracja 6000, Strata: 3.2432
Iteracja 6500, Strata: 3.9161
Iteracja 7000, Strata: 2.2851
Iteracja 7500, Strata: 1.1089
Iteracja 8000, Strata: 0.5071
Iteracja 8500, Strata: 0.3045
Iteracja 9000, Strata: 1.9144
Iteracja 9500, Strata: 1.0655
Iteracja 10000, Strata: 1.9189
Iteracja 10500, Strata: 0.2414
Iteracja 11000, Strata: 0.7321
Iteracja 11500, Strata: 0.1890
Iteracja 12000, Strata: 1.8771
Iteracja 12500, Strata: 1.6049
Iteracja 13000, Strata: 0.8099
Iteracja 13500, Strata: 0.1295
Osiągnięto wymaganą dokładność (strata: 0.0999) w iteracji 13736
